In [22]:
import pandas as pd
from IPython.display import Image, HTML

In [23]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [24]:
path = "sample1.csv" # Input CSV
data = pd.read_csv(path)
games = pd.read_csv('games.csv').loc[:,['name','slug']]
trans_games = games.set_index('name').T

In [25]:
# Add images
img_dict = trans_games.to_dict('list')
images = []
for i in data['name']:
    images.append("https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg")
data["pic"] = images
pd.set_option('display.max_colwidth', -1)

C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [26]:
# Change index
data.index += 1

In [27]:
pre = "<head>\n<script src=\"https://kryogenix.org/code/browser/sorttable/sorttable.js\"></script>\n</head>\n\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n</div>\n\n"
post = (
    "\n"
    "\n<script src=\"main.js\"></script>"
    "\n</body>"
    )

In [28]:
# render dataframe as html
html = HTML(data.to_html(escape=False ,formatters=dict(pic=path_to_image_html)))
body = html.data
body = body.replace("dataframe", "sortable", 1)
body = body.replace("class=\"sortable\"", "id=\"tbl\" class=\"sortable\"", 1)
a = pre + body + post
# write html to file
with open('result.html', 'w') as f:
    f.write(a)